<a href="https://colab.research.google.com/github/b4mbanh/Tugas-Social-media-analyst/blob/main/Welcome_to_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
from tqdm import tqdm # Library untuk membuat progress bar

# HEADERS digunakan untuk "menyamar" sebagai browser asli.
# Ini penting agar permintaan kita tidak mudah diblokir oleh server website.
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def scrape_article_detail(url):
    """
    Fungsi ini bertugas mengunjungi satu halaman artikel spesifik.
    Tujuannya untuk mengambil data yang lebih detail: isi berita, penulis, dan tanggal.
    """
    try:
        # 1. Kirim permintaan HTTP ke URL artikel
        response = requests.get(url, headers=HEADERS, timeout=10)
        # Jika halaman tidak ditemukan atau ada error, fungsi berhenti.
        if response.status_code != 200:
            return None

        # 2. Parse HTML halaman dengan BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # 3. Cari elemen-elemen spesifik berdasarkan class CSS-nya
        content_div = soup.find('div', class_='detail__body-text')
        if not content_div: # Jika struktur konten tidak ada, hentikan.
            return None

        # Ambil semua paragraf <p> di dalam div konten dan gabungkan.
        paragraphs = content_div.find_all('p')
        full_content = ' '.join([p.text.strip() for p in paragraphs])

        author_elem = soup.find('div', class_='detail__author')
        author = author_elem.text.strip() if author_elem else 'N/A'

        date_elem = soup.find('div', class_='detail__date')
        published_date = date_elem.text.strip() if date_elem else 'N/A'

        # 4. Kembalikan data detail dalam bentuk dictionary
        return {
            'author': author,
            'published_date': published_date,
            'content': full_content
        }
    except Exception as e:
        # Jika terjadi error apapun (misal: koneksi putus), fungsi berhenti.
        return None

def scrape_detik_indeks(total_pages=10):
    """
    Fungsi utama yang menjadi "manajer" proses scraping.
    Fungsi ini akan menjelajahi halaman-halaman indeks berita.
    """
    all_articles = [] # List kosong untuk menampung semua hasil scraping
    base_url = 'https://news.detik.com'

    print(f"Memulai scraping {total_pages} halaman dari Detik News...")

    # Looping untuk setiap nomor halaman (dari 1 sampai total_pages)
    for page in range(1, total_pages + 1):
        # Logika URL: halaman 1 tidak pakai parameter, halaman 2 dst pakai ?page=...
        if page == 1:
            index_url = f"{base_url}/indeks"
        else:
            index_url = f"{base_url}/indeks?page={page}"

        print(f"\nScraping Halaman Indeks #{page}: {index_url}")

        try:
            # Kunjungi halaman indeks
            response = requests.get(index_url, headers=HEADERS, timeout=10)
            if response.status_code != 200:
                print(f"Gagal mengakses halaman indeks #{page}. Melanjutkan...")
                continue # Lanjut ke halaman berikutnya

            soup = BeautifulSoup(response.text, 'html.parser')

            # Cari semua "bungkus" artikel, yaitu tag <article>
            articles_on_page = soup.find_all('article')

            if not articles_on_page:
                print(f"Tidak ada artikel ditemukan di halaman {page}.")
                continue

            # Looping untuk setiap artikel yang ditemukan di halaman indeks
            # tqdm() akan otomatis membuat progress bar di terminal
            for article_tag in tqdm(articles_on_page, desc=f"Memproses Artikel Halaman {page}"):

                link_tag = article_tag.find('a')
                if not link_tag or not link_tag.has_attr('href'):
                    continue

                article_url = link_tag['href']
                if not article_url.startswith('http'):
                    continue

                # Ambil metadata awal dari halaman indeks
                title_tag = article_tag.find('h3', class_='media__title')
                title = title_tag.text.strip() if title_tag else 'N/A'

                img_tag = article_tag.find('img')
                image_url = img_tag['src'] if img_tag and img_tag.has_attr('src') else 'N/A'

                category_tag = article_tag.find('span', class_='media__nav__item')
                category = category_tag.text.strip() if category_tag else 'N/A'

                # Panggil fungsi detail untuk mengambil isi berita, penulis, dll.
                detail_data = scrape_article_detail(article_url)

                # Jika data detail berhasil didapatkan, gabungkan semuanya
                if detail_data:
                    article_data = {
                        'title': title,
                        'category': category,
                        'published_date': detail_data['published_date'],
                        'author': detail_data['author'],
                        'article_url': article_url,
                        'image_url': image_url,
                        'content': detail_data['content'],
                        'source_page': page
                    }
                    # Masukkan data lengkap ke dalam list utama
                    all_articles.append(article_data)

                # Beri jeda 0.5 detik, ini etika scraping agar tidak membebani server
                time.sleep(0.5)

        except Exception as e:
            print(f"Terjadi error tak terduga di halaman {page}: {e}")
            continue

    return all_articles

# Ini adalah titik awal program dieksekusi
if __name__ == "__main__":
    # --- PENGATURAN ---
    # Atur jumlah halaman yang ingin di-scrape. 10 halaman akan menghasilkan sekitar 150-200 baris.
    # Ini jumlah yang ideal untuk menunjukkan kemampuan skrip tanpa menunggu terlalu lama.
    JUMLAH_HALAMAN_SCRAPE = 10

    # --- EKSEKUSI ---
    scraped_data = scrape_detik_indeks(total_pages=JUMLAH_HALAMAN_SCRAPE)

    # --- EKSPOR DATA ---
    if scraped_data:
        print(f"\nScraping Selesai! Total {len(scraped_data)} artikel berhasil dikumpulkan.")

        # Gunakan Pandas untuk mengubah list data menjadi format tabel
        df = pd.DataFrame(scraped_data)

        # Simpan ke file CSV (bisa dibuka di Excel)
        csv_filename = 'hasil_scrape_detik.csv'
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        print(f"Data berhasil diekspor ke file CSV: {csv_filename}")

        # Simpan ke file JSON
        json_filename = 'hasil_scrape_detik.json'
        df.to_json(json_filename, orient='records', lines=True, force_ascii=False, indent=4)
        print(f"Data berhasil diekspor ke file JSON: {json_filename}")

        print("\n--- Preview 5 Data Pertama ---")
        print(df.head())

    else:
        print("\nTidak ada data yang berhasil di-scrape. Cek koneksi atau struktur website mungkin telah berubah.")


Memulai scraping 10 halaman dari Detik News...

Scraping Halaman Indeks #1: https://news.detik.com/indeks


Memproses Artikel Halaman 1: 100%|██████████| 20/20 [00:36<00:00,  1.83s/it]



Scraping Halaman Indeks #2: https://news.detik.com/indeks?page=2


Memproses Artikel Halaman 2: 100%|██████████| 20/20 [00:36<00:00,  1.84s/it]



Scraping Halaman Indeks #3: https://news.detik.com/indeks?page=3


...


Scraping Halaman Indeks #9: https://news.detik.com/indeks?page=9


Memproses Artikel Halaman 9: 100%|██████████| 20/20 [00:37<00:00,  1.87s/it]



Scraping Halaman Indeks #10: https://news.detik.com/indeks?page=10


Memproses Artikel Halaman 10: 100%|██████████| 20/20 [00:37<00:00,  1.88s/it]


Scraping Selesai! Total 183 artikel berhasil dikumpulkan.
Data berhasil diekspor ke file CSV: hasil_scrape_detik.csv
Data berhasil diekspor ke file JSON: hasil_scrape_detik.json

--- Preview 5 Data Pertama ---
                                               title category  \
0  Mengenal Kategori Siklon Tropis, dari Terlemah...      N/A   
1  Kapolda Riau Tegaskan Green Policing Jadi Pila...      N/A   
2  Kronologi Tukang Parkir Pukul Pemotor di Jakut...      N/A   
3  Daftar Fenomena Astronomi di Oktober 2025 dan ...      N/A   
4  Pidato Prabowo di PBB: Kemerdekaan Palestina d...      N/A   

                  published_date                            author  \
0  Minggu, 28 Sep 2025 19:35 WIB  Widhia Arum Wibawana - detikNews   
1  Minggu, 28 Sep 2025 19:27 WIB          Mei Amelia R - detikNews   
2  Minggu, 28 Sep 2025 19:23 WIB   Rizky Adha Mahendra - detikNews   
3  Minggu, 28 Sep 2025 19:19 WIB  Widhia Arum Wibawana - detikNews   
4  Minggu, 28 Sep 2025 19:03 WIB                